<a href="https://colab.research.google.com/github/fantajeon/DLPytorch1.2/blob/master/Chapter1_MLP_Criteo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Criteo MLP #
Author: fantajeon@gmail.com

Criteo는 Kaggle 사이트통해서 공개한 데이터 셋입니다. 이 데이터의 목적은 사용자의 click을 예측하는 데이터 셋입니다. 라벨과 특징으로 구성되어 있습니다. 이 데이터셋의 특징은 특징에 대한 특별한 설명을 하지 않습니다. 보안때문이라고 이야기 했습니다. 그래서 딥러닝 개발자들이 즐겨 쓰는 데이터셋입니다. 아무런 도메인 설명도 하지 않았지만, 성능을 내기 때문입니다.
최신 트렌드 [성능 표](https://paperswithcode.com/paper/xdeepfm-combining-explicit-and-implicit#code)를 참조하세요.

라벨: 
*   1 - 클릭했다.
*   0 - 클릭 안 했다.

특징:
* I1-I13: 연속 특징
* C1-C24: 카테고리 특징

In [69]:
!pip install --upgrade torch==1.2.0
!pip install torchnet


Requirement already up-to-date: torch==1.2.0 in /usr/local/lib/python3.6/dist-packages (1.2.0)
     |████████████████████████████████| 686kB 7.9MB/s 
     |████████████████████████████████| 204kB 45.8MB/s 
  Created wheel for torchnet: filename=torchnet-0.0.4-cp36-none-any.whl size=29743 sha256=3a808bce518a7ce95ab1ebd7bee3d0947680ca73cbe662cc43ba78bfac50de09
  Stored in directory: /root/.cache/pip/wheels/e1/03/fb/1c212c2f20905cdf97fe39022946cf16b8e66ed754a6663400
  Created wheel for visdom: filename=visdom-0.1.8.9-cp36-none-any.whl size=655252 sha256=e1f3616ffbe99fec7e5156cf9e94317401e7dc99382739c8560b49e14f18d313
  Stored in directory: /root/.cache/pip/wheels/70/19/a7/6d589ed967f4dfefd33bc166d081257bd4ed0cb618dccfd62a
  Created wheel for torchfile: filename=torchfile-0.1.0-cp36-none-any.whl size=5711 sha256=40183ce2e7ebe4504d05c2983dcff22c2ce86edb8437e9d9b1dd145df3c7a670
  Stored in directory: /root/.cache/pip/wheels/b1/c3/d6/9a1cc8f3a99a0fc1124cae20153f36af59a6e683daca0a0814
Successf

In [70]:
import torch
from torch import nn
import torch.optim
import torch.nn.functional as F
import pandas as pd
import numpy as np
import torch.utils.data as data
from IPython.core.debugger import set_trace
import torchnet.meter as meter
torch.__version__


'1.2.0'

In [0]:
!wget -q https://raw.githubusercontent.com/ycjuan/kaggle-2014-criteo/master/train.tiny.csv -O ./train.csv
!wget -q https://github.com/fantajeon/DLPytorch1.2_Materials/raw/master/test_tiny.csv -O ./test.csv

In [99]:
test_data = pd.read_csv("test.csv", header=None)
train_data = pd.read_csv("train.csv")
train_data = train_data.iloc[:,1:]
print("len(train)", train_data.shape, "len(test)", test_data.shape)
train_data.head(5)


len(train) (1999, 40) len(test) (10000, 40)


,Label,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,I11,I12,I13,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,0,1.0,1,5.0,0.0,1382.0,4.0,15.0,2.0,181.0,1.0,2.0,NaN,2.0,68fd1e64,80e26c9b,fb936136,7b4723c4,25c83c98,7e0ccccf,de7995b8,1f89b562,a73ee510,a8cd5504,b2cb9c98,37c9c164,2824a5f6,1adce6ef,8ba8b39a,891b62e7,e5ba7672,f54016b9,21ddcdc9,b1252a9d,07b5194c,NaN,3a171ecb,c5c50484,e8b83407,9727dd16
1,0,2.0,0,44.0,1.0,102.0,8.0,2.0,2.0,4.0,1.0,1.0,NaN,4.0,68fd1e64,f0cf0024,6f67f7e5,41274cd7,25c83c98,fe6b92e5,922afcc0,0b153874,a73ee510,2b53e5fb,4f1b46f3,623049e6,d7020589,b28479f6,e6c5b5cd,c92f3b61,07c540c4,b04e4670,21ddcdc9,5840adea,60f6221e,NaN,3a171ecb,43f13e8b,e8b83407,731c3655
2,0,2.0,0,1.0,14.0,767.0,89.0,4.0,2.0,245.0,1.0,3.0,3.0,45.0,287e684f,0a519c5c,02cf9876,c18be181,25c83c98,7e0ccccf,c78204a1,0b153874,a73ee510,3b08e48b,5f5e6091,8fe001f4,aa655a2f,07d13a8f,6dc710ed,36103458,8efede7f,3412118d,NaN,NaN,e587c466,ad3062eb,3a171ecb,3b183c5c,NaN,NaN
3,0,NaN,893,NaN,NaN,4392.0,NaN,0.0,0.0,0.0,NaN,0.0,NaN,NaN,68fd1e64,2c16a946,a9a87e68,2e17d6f6,25c83c98,fe6b92e5,2e8a689b,0b153874,a73ee510,efea433b,e51ddf94,a30567ca,3516f6e6,07d13a8f,18231224,52b8680f,1e88c74f,74ef3502,NaN,NaN,6b3a5ca6,NaN,3a171ecb,9117a34a,NaN,NaN
4,0,3.0,-1,NaN,0.0,2.0,0.0,3.0,0.0,0.0,1.0,1.0,NaN,0.0,8cf07265,ae46a29d,c81688bb,f922efad,25c83c98,13718bbd,ad9fa255,0b153874,a73ee510,5282c137,e5d8af57,66a76a26,f06c53ac,1adce6ef,8ff4b403,01adbab4,1e88c74f,26b3c7a7,NaN,NaN,21c9516a,NaN,32c7478e,b34f3128,NaN,NaN


In [96]:
test_data.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39
0,1,NaN,0,4.0,1.0,17714.0,66.0,2.0,11.0,62.0,NaN,1.0,NaN,1.0,24eda356,207b2d81,6a061171,ef6ae422,25c83c98,6f6d9be8,b5043bcd,0b153874,a73ee510,b78f3a55,371dae82,2dad7b23,18fc2b1e,07d13a8f,0c67c4ca,722ee340,d4bb7bd8,395856b0,21ddcdc9,b1252a9d,aec73e74,NaN,32c7478e,215dcc2f,001f3601,05051329
1,0,10.0,12,19.0,30.0,4.0,30.0,10.0,30.0,30.0,1.0,1.0,NaN,30.0,05db9164,e5fb1af3,50f62395,e3b436dd,4cf72387,7e0ccccf,095d6b9e,0b153874,a73ee510,94c7e002,8f736c02,b735321f,954f731f,b28479f6,23287566,76a47423,e5ba7672,13145934,0053530c,b1252a9d,d4815352,NaN,bcdee96c,2ef19589,f7839e21,eadff93b
2,1,NaN,6,116.0,10.0,27329.0,NaN,0.0,29.0,1.0,NaN,0.0,NaN,22.0,8cf07265,73a46ff0,dd1e882e,f34203e3,25c83c98,7e0ccccf,3b870555,5b392875,a73ee510,3b08e48b,39ec7a77,0a48382f,66bf004c,b28479f6,4f648a87,4e669708,776ce399,da507f45,21ddcdc9,5840adea,81f8acaf,NaN,be7c41b4,15b4ecfa,ea9a246c,9636866f
3,0,NaN,-1,NaN,NaN,5713.0,NaN,0.0,1.0,14.0,NaN,0.0,NaN,NaN,05db9164,4c2bc594,d032c263,c18be181,25c83c98,fe6b92e5,646b4bca,0b153874,a73ee510,3b08e48b,45ea640a,dfbb09fb,df941d33,8ceecbc8,c7b09696,84898b2a,d4bb7bd8,fe94fad1,NaN,NaN,0014c32a,ad3062eb,3a171ecb,3b183c5c,NaN,NaN
4,0,0.0,1,NaN,10.0,28841.0,NaN,0.0,10.0,5.0,0.0,0.0,NaN,10.0,05db9164,b46aceb6,NaN,NaN,b0530c50,7e0ccccf,b5e1898d,0b153874,a73ee510,3b08e48b,5d1b7285,NaN,cb2e33ed,07d13a8f,217d99f2,NaN,07c540c4,908eaeb8,NaN,NaN,NaN,NaN,be7c41b4,NaN,NaN,NaN


#연속필드만으로 예측#

In [0]:
class MLP(torch.nn.Module):
  def __init__(self, input_dim, hidden_dims, output_dim):
    super(MLP, self).__init__()
    self.input_dim = input_dim
    self.output_dim = output_dim
    layers = []
    ndims = [self.input_dim] + hidden_dims + [output_dim]
    for i in range(0, len(hidden_dims)+1):
      seq = nn.Sequential( 
          nn.Linear(ndims[i], ndims[i+1]),
          nn.BatchNorm1d(ndims[i+1]),
          nn.ReLU()
      )
      layers.append( seq )
    self.layers = nn.ModuleList( layers )
    self.init_weight()

  def forward(self, x):
    for i, layer in enumerate(self.layers):
      x = layer(x)
    return x

  def init_weight(self):
    for m in self.modules():
      if isinstance(m, nn.Linear):
        nn.init.kaiming_normal_(m.weight, mode='fan_in')
        nn.init.constant_(m.bias, 0.)
      elif isinstance(m, nn.BatchNorm1d):
        nn.init.constant_(m.weight, 1.)
        nn.init.constant_(m.bias, 0.)

## 데이터 정제 ##

In [0]:
def preprocess(data):
  # NaN은 의미있는 숫자가 아니므로 계산을 할 수 없습니다. 여러 방법이 있지만, 평균값을 채워 넣는다.
  c_label = data.iloc[:,0].values
  c_data = data.iloc[:,1:14].values
  for i in range(c_data.shape[1]):
    invalid_idx = np.isnan(c_data[:,i])
    mean = c_data[~invalid_idx,i].mean()   # NaN아닌 데이터의 평균 계산
    c_data[invalid_idx,i] = mean  # NaN을 평균값으로 대체

  c_data = torch.tensor(c_data, dtype=torch.float32)
  c_label = torch.tensor(c_label, dtype=torch.int64)  
  return c_data, c_label

### Train Data Set 정제 ###

In [0]:
c_train_data, c_train_label = preprocess(train_data)
ds = data.TensorDataset(c_train_data, c_train_label )
train_loader = data.DataLoader(ds, batch_size=128, shuffle=True)

In [102]:
c_train_label.sum()

tensor(418)

### Test Data Set 정제 ###

In [0]:
c_test_data, c_test_label = preprocess(test_data)
ds = data.TensorDataset(c_test_data, c_test_label )
test_loader = data.DataLoader(ds, batch_size=1000, shuffle=True)

In [104]:
c_test_label.sum()

tensor(2608)

## Loss 함수 설계 ##
Cross Entropy Loss 함수 설계
$$Loss(y,\hat{y}) = \sum_{c \in Classes} { -y_c * log(\hat{y_c}) }$$

True $y$ 설계, One-Hot Encoding 기법(하나만 1인 형태)
* label이 1일 경우, y = [0, 1]  
* label이 0일 경우, y = [1, 0]

예측 $\hat{y}$의 설계
* posivie 예측일 경우, $\hat{y}$ = [0.01, 0.99]
* 그렇지 않을 경우, $\hat{y}$ = [0.99, 0.01]

그런데, 내장된 함수가 있습니다. Torch Manual에서 [loss](https://pytorch.org/docs/stable/nn.html#loss-functions) 편을 참조 하시기 바랍니다. 많이 존재합니다. torch.nn.BCEWithLogitsLoss()입니다.

In [12]:
def one_hot_encoder(y, nclass):
  label = torch.zeros(y.size(0), nclass, dtype=torch.float32)
  label.scatter_(1,y.unsqueeze(1),1.0)
  return label

nclass = 2
y = torch.tensor([1,0,1,1,0])
one_hot_encoder(y,2)

tensor([[0., 1.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [1., 0.]])

In [44]:
loss = torch.nn.BCEWithLogitsLoss()

true_y = torch.tensor( [0,1,0] )
pred_y = torch.tensor( [[0.1, 0.9], [0.2, 0.8], [0.9, 0.1]], dtype=torch.float32 )
label_y = one_hot_encoder(true_y, 2)

loss(label_y, pred_y)

tensor(0.7032)

### 정확도 계산 ###


In [0]:
def check_accuracy(model, dataloader, mtr):
  x, true_y = next(dataloader)
  with torch.no_grad():
    logit_y = model(x)
    cls_prob = F.softmax(logit_y,dim=1)
    pred_y = cls_prob.argmax(dim=1)
    acc = (pred_y == true_y).type(dtype=torch.float32).mean()
    mtr.add(cls_prob[:,1], true_y)
  return acc


# 이제 학습을 시작해 보자 #

In [106]:


model = MLP(13,[200,200,200], 2)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
#optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
loss = torch.nn.BCEWithLogitsLoss()
mtr = meter.AUCMeter()


test_iter = iter(test_loader)
global_step = 0
for epoch in range(2000):
  for i, (x, y) in enumerate(loader):
    global_step += 1
    pred_y = model(x)
    label_y = one_hot_encoder(y, nclass=2)
    err = loss(pred_y, label_y)
    optimizer.zero_grad()
    err.backward()
    optimizer.step()
    if global_step < 10 or global_step% 1000 == 0:
      model.eval()
      try:
        acc = check_accuracy(model, test_iter, mtr)
        print("epoch", epoch, "err", err.item(), "acc", acc.item()*100.0, "AUC", mtr.value()[0])
      except StopIteration:
        # 끝까지 다 했을 경우, 다시 처음부터 반복
        test_iter = iter(test_loader)
      model.train()

epoch 0 err 0.7076781988143921 acc 58.99999737739563 AUC 0.5955260562768572
epoch 0 err 0.6855890154838562 acc 57.099997997283936 AUC 0.5955309909198578
epoch 0 err 0.6870840787887573 acc 54.500001668930054 AUC 0.5882522935753304
epoch 0 err 0.675654947757721 acc 56.09999895095825 AUC 0.5957120965323164
epoch 0 err 0.6906782388687134 acc 61.799997091293335 AUC 0.6034235539786255
epoch 0 err 0.6743243932723999 acc 64.3999993801117 AUC 0.6109761203511204
epoch 0 err 0.682582676410675 acc 70.39999961853027 AUC 0.6168308125462438
epoch 0 err 0.673927903175354 acc 69.80000138282776 AUC 0.6145621574370582
epoch 0 err 0.6822398900985718 acc 71.79999947547913 AUC 0.6208388563699406
epoch 62 err 0.5906600952148438 acc 71.60000205039978 AUC 0.606020146137094
epoch 187 err 0.548851728439331 acc 74.50000047683716 AUC 0.6011630687735449
epoch 249 err 0.5777088403701782 acc 74.29999709129333 AUC 0.5997931109887905
epoch 312 err 0.5563284754753113 acc 75.59999823570251 AUC 0.6003230652872509
epoch 37